In [11]:
# Configuration
from pathlib import Path
from utils.database.db_utils import get_db_connection
 
# Define the path to the DuckDB database file
project_root = Path.cwd().parent if "notebooks" in str(Path.cwd()) else Path.cwd()
db_path = project_root / "data" / "processed" / "chess_games.db"
 
# Define partitions for player_opening_stats
partitions = list("ABCDE") + ["other"]
 
# Print configuration details
print(f"Database path: {db_path}")
print(f"Partitions: {partitions}")

Database path: /Users/a/Documents/personalprojects/chess-opening-recommender/data/processed/chess_games.db
Partitions: ['A', 'B', 'C', 'D', 'E', 'other']


# Purpose

To decrease the size of our opening stats DB, making queries faster and more efficient.

# Methods:

    1. Change INTs to SMALLINT
        - player_id and opening_id
        - num_draws
            - Not doing this on num_wins and num_losses because those may exceed the limit of this data type (~32,000)
        - Not doing this with player_opening_stats ID because that's a composite string, not an int 
        - Saves 2 bytes per entry
    
    2. Change `color` VARCHAR to w/b enum
        - 1b
        - Compared to the 12-16bytes the previous varchar took up

# Considerations
    - Saving a copy of the DB in case I bork this

In [7]:
# Print current database size
import os
try:
    db_size_bytes = os.path.getsize(db_path)
    db_size_mb = db_size_bytes / (1024 * 1024)
    print(f"Current database file size: {db_size_mb:.2f} MB")
except FileNotFoundError:
    print("Database file not found.")
    db_size_mb = 0

Current database file size: 1385.51 MB


In [14]:
# Diagnostic - check actual table schemas and data counts
with get_db_connection(db_path) as con:
    print("=== CURRENT TABLE SCHEMAS ===")
    
    # Check player table
    player_schema = con.execute("DESCRIBE player").fetchall()
    print(f"\nPLAYER table schema:")
    for col in player_schema:
        print(f"  {col[0]}: {col[1]}")
    
    # Check opening table  
    opening_schema = con.execute("DESCRIBE opening").fetchall()
    print(f"\nOPENING table schema:")
    for col in opening_schema:
        print(f"  {col[0]}: {col[1]}")
    
    # Check one stats table
    stats_schema = con.execute("DESCRIBE player_opening_stats_A").fetchall()
    print(f"\nPLAYER_OPENING_STATS_A table schema:")
    for col in stats_schema:
        print(f"  {col[0]}: {col[1]}")
    
    print("\n=== DATA RANGES ===")
    
    # Check max values
    max_player_id = con.execute("SELECT MAX(id) FROM player").fetchone()[0]
    max_opening_id = con.execute("SELECT MAX(id) FROM opening").fetchone()[0]
    print(f"Max player_id: {max_player_id:,}")
    print(f"Max opening_id: {max_opening_id:,}")
    
    # Check total row counts
    total_stats_rows = 0
    for letter in ['A', 'B', 'C', 'D', 'E', 'other']:
        count = con.execute(f"SELECT COUNT(*) FROM player_opening_stats_{letter}").fetchone()[0]
        print(f"player_opening_stats_{letter}: {count:,} rows")
        total_stats_rows += count
    
    print(f"\nTotal stats rows: {total_stats_rows:,}")
    
    # Check color values distribution
    color_dist = con.execute("SELECT color, COUNT(*) FROM player_opening_stats_A GROUP BY color").fetchall()
    print(f"\nColor distribution in stats_A: {color_dist}")
    
    # Check num_draws distribution
    draws_stats = con.execute("SELECT MIN(num_draws), MAX(num_draws), AVG(num_draws) FROM player_opening_stats_A").fetchone()
    print(f"\nnum_draws in stats_A - Min: {draws_stats[0]}, Max: {draws_stats[1]}, Avg: {draws_stats[2]:.2f}")

=== CURRENT TABLE SCHEMAS ===

PLAYER table schema:
  id: INTEGER
  name: VARCHAR
  title: VARCHAR

OPENING table schema:
  id: INTEGER
  eco: VARCHAR
  name: VARCHAR

PLAYER_OPENING_STATS_A table schema:
  player_id: INTEGER
  opening_id: INTEGER
  color: ENUM('w', 'b')
  num_wins: INTEGER
  num_draws: SMALLINT
  num_losses: INTEGER

=== DATA RANGES ===
Max player_id: 32,964,861
Max opening_id: 5,222,927
player_opening_stats_A: 3,137,793 rows
player_opening_stats_B: 3,339,867 rows
player_opening_stats_C: 4,175,489 rows
player_opening_stats_D: 1,771,597 rows
player_opening_stats_E: 442,866 rows
player_opening_stats_other: 0 rows

Total stats rows: 12,867,612

Color distribution in stats_A: [('w', 1108563), ('b', 2029230)]

num_draws in stats_A - Min: 0, Max: 480, Avg: 0.45


In [ ]:
# Execute PROPER database shrinking - compact the wasteful IDs
if db_size_mb > 0:
    with get_db_connection(db_path) as con:
        print("=== ANALYZING THE WASTE ====")
        player_stats = con.execute('SELECT MIN(id), MAX(id), COUNT(*) FROM player').fetchone()
        opening_stats = con.execute('SELECT MIN(id), MAX(id), COUNT(*) FROM opening').fetchone()
        
        print(f'Players: {player_stats[2]:,} records, IDs from {player_stats[0]:,} to {player_stats[1]:,}')
        print(f'Openings: {opening_stats[2]:,} records, IDs from {opening_stats[0]:,} to {opening_stats[1]:,}')
        print(f'Player ID waste: {((player_stats[1] - player_stats[2]) / player_stats[1] * 100):.1f}%')
        print(f'Opening ID waste: {((opening_stats[1] - opening_stats[2]) / opening_stats[1] * 100):.1f}%')
        
        print("\n=== COMPACTING DATABASE WITH SEQUENTIAL IDS ===")
        con.execute("CREATE TYPE IF NOT EXISTS color_enum AS ENUM ('w', 'b');")

        # Create new player table with compact sequential IDs
        print("Creating compacted player table...")
        con.execute("""
            CREATE TABLE player_new AS
            SELECT 
                ROW_NUMBER() OVER (ORDER BY name) as id,
                name,
                title
            FROM player;
        """)
        
        # Create new opening table with compact sequential IDs  
        print("Creating compacted opening table...")
        con.execute("""
            CREATE TABLE opening_new AS
            SELECT 
                ROW_NUMBER() OVER (ORDER BY eco, name) as id,
                eco,
                name
            FROM opening;
        """)
        
        # Create mapping tables to translate old IDs to new IDs
        print("Creating ID mapping tables...")
        con.execute("""
            CREATE TEMP TABLE player_id_mapping AS
            SELECT p_old.id as old_id, p_new.id as new_id, p_old.name
            FROM player p_old
            JOIN player_new p_new ON p_old.name = p_new.name;
        """)
        
        con.execute("""
            CREATE TEMP TABLE opening_id_mapping AS
            SELECT o_old.id as old_id, o_new.id as new_id, o_old.eco, o_old.name
            FROM opening o_old
            JOIN opening_new o_new ON o_old.eco = o_new.eco AND o_old.name = o_new.name;
        """)
        
        # Verify the new ID ranges
        new_player_stats = con.execute('SELECT MIN(id), MAX(id), COUNT(*) FROM player_new').fetchone()
        new_opening_stats = con.execute('SELECT MIN(id), MAX(id), COUNT(*) FROM opening_new').fetchone()
        
        print(f'\nNew player IDs: {new_player_stats[0]:,} to {new_player_stats[1]:,} ({new_player_stats[2]:,} records)')
        print(f'New opening IDs: {new_opening_stats[0]:,} to {new_opening_stats[1]:,} ({new_opening_stats[2]:,} records)')
        
        # Check if we can use SMALLINT (max 32,767) - player_id needs INTEGER, opening_id can be SMALLINT
        can_use_smallint_opening = new_opening_stats[1] <= 32767
        can_use_smallint_player = new_player_stats[1] <= 32767
        
        player_id_type = "SMALLINT" if can_use_smallint_player else "INTEGER"
        opening_id_type = "SMALLINT" if can_use_smallint_opening else "INTEGER"
        
        print(f'\nPlayer IDs can use SMALLINT: {can_use_smallint_player} -> Using {player_id_type}')
        print(f'Opening IDs can use SMALLINT: {can_use_smallint_opening} -> Using {opening_id_type}')

        # Create optimized stats tables with compacted IDs
        print("\nCreating optimized stats tables with compacted IDs...")
        for letter in partitions:
            new_table = f"player_opening_stats_{letter}_new"
            print(f"Creating {new_table}...")
            
            con.execute(f"""
                CREATE TABLE {new_table} (
                    player_id   {player_id_type} NOT NULL,
                    opening_id  {opening_id_type} NOT NULL,
                    color       color_enum NOT NULL,
                    num_wins    INTEGER DEFAULT 0,
                    num_draws   SMALLINT DEFAULT 0,
                    num_losses  INTEGER DEFAULT 0,
                    PRIMARY KEY (player_id, opening_id, color)
                );
            """)
            
            # Migrate data with new compacted IDs and optimized types
            old_table = f"player_opening_stats_{letter}"
            print(f"Migrating {old_table} with compacted IDs...")
            
            con.execute(f"""
                INSERT INTO {new_table} (player_id, opening_id, color, num_wins, num_draws, num_losses)
                SELECT 
                    CAST(pm.new_id AS {player_id_type}) as player_id,
                    CAST(om.new_id AS {opening_id_type}) as opening_id,
                    CASE 
                        WHEN s.color = 'w' THEN 'w'::color_enum
                        ELSE 'b'::color_enum
                    END as color,
                    s.num_wins,
                    CAST(s.num_draws AS SMALLINT) as num_draws,
                    s.num_losses
                FROM {old_table} s
                JOIN player_id_mapping pm ON s.player_id = pm.old_id
                JOIN opening_id_mapping om ON s.opening_id = om.old_id
                WHERE s.num_wins > 0 OR s.num_draws > 0 OR s.num_losses > 0;
            """)

        # Now swap the tables
        print("\nSwapping old tables with optimized new tables...")
        
        # Drop old tables
        for letter in partitions:
            con.execute(f"DROP TABLE player_opening_stats_{letter};")
        con.execute("DROP TABLE player;")
        con.execute("DROP TABLE opening;")
        
        # Rename new tables
        for letter in partitions:
            con.execute(f"ALTER TABLE player_opening_stats_{letter}_new RENAME TO player_opening_stats_{letter};")
        con.execute("ALTER TABLE player_new RENAME TO player;")
        con.execute("ALTER TABLE opening_new RENAME TO opening;")
        
        # Reset sequences to reasonable values
        print("\nResetting sequences to proper values...")
        con.execute(f"DROP SEQUENCE IF EXISTS player_id_seq;")
        con.execute(f"DROP SEQUENCE IF EXISTS opening_id_seq;")
        con.execute(f"CREATE SEQUENCE player_id_seq START {new_player_stats[1] + 1};")
        con.execute(f"CREATE SEQUENCE opening_id_seq START {new_opening_stats[1] + 1};")
        
        # Recreate the view
        union_selects = "\nUNION ALL\n".join([
            f"SELECT * FROM player_opening_stats_{letter}"
            for letter in partitions
        ])
        con.execute(f"""
            CREATE OR REPLACE VIEW player_opening_stats AS
            {union_selects};
        """)
        
        print("\n=== COMPACTION COMPLETE ===")
        final_player_stats = con.execute('SELECT MIN(id), MAX(id), COUNT(*) FROM player').fetchone()
        final_opening_stats = con.execute('SELECT MIN(id), MAX(id), COUNT(*) FROM opening').fetchone()
        
        print(f'Final player IDs: {final_player_stats[0]:,} to {final_player_stats[1]:,}')
        print(f'Final opening IDs: {final_opening_stats[0]:,} to {final_opening_stats[1]:,}')
        print(f'Space savings: Player IDs now use {player_id_type}, Opening IDs use {opening_id_type}')
        print(f'Color values now use 1-byte ENUM instead of VARCHAR')
        print(f'num_draws now uses SMALLINT instead of INTEGER')
        print("Removed all zero-value records")
        
        # Count final records
        total_final_records = 0
        for letter in partitions:
            count = con.execute(f'SELECT COUNT(*) FROM player_opening_stats_{letter}').fetchone()[0]
            total_final_records += count
        print(f'\nTotal optimized records: {total_final_records:,}')

Max opening_id: 5222927
Creating new schema for partitioned tables...
Creating new opening table with optimized ID type...
Migrating data to new opening table...
Creating new player_opening_stats tables with optimized data types...
Migrating player_opening_stats data with optimized types...
Migrating data from player_opening_stats_A to player_opening_stats_A_new...
Migrating data from player_opening_stats_B to player_opening_stats_B_new...
Migrating data from player_opening_stats_B to player_opening_stats_B_new...
Migrating data from player_opening_stats_C to player_opening_stats_C_new...
Migrating data from player_opening_stats_C to player_opening_stats_C_new...
Migrating data from player_opening_stats_D to player_opening_stats_D_new...
Migrating data from player_opening_stats_D to player_opening_stats_D_new...
Migrating data from player_opening_stats_E to player_opening_stats_E_new...
Migrating data from player_opening_stats_E to player_opening_stats_E_new...
Migrating data from play

In [9]:
# Print new database size and vacuum
try:
    db_size_bytes = os.path.getsize(db_path)
    db_size_mb = db_size_bytes / (1024 * 1024)
    print(f"New database file size: {db_size_mb:.2f} MB")
 
    with get_db_connection(db_path) as con:
        print("Vacuuming database to optimize storage...")
        con.execute("VACUUM;")
        print("Vacuum complete.")
 
        db_size_bytes = os.path.getsize(db_path)
        db_size_mb = db_size_bytes / (1024 * 1024)
        print(f"Post-vacuum database file size: {db_size_mb:.2f} MB")
except FileNotFoundError:
    print("Database file not found.")

New database file size: 1722.51 MB
Vacuuming database to optimize storage...
Vacuum complete.
Post-vacuum database file size: 1722.51 MB


In [10]:
# Force DuckDB to rewrite the file to reclaim space
with get_db_connection(db_path) as con:
    print("Forcing database file rewrite to reclaim space...")
    con.execute("CHECKPOINT;")
    con.execute("PRAGMA force_checkpoint;")
    print("Checkpoint complete.")

# Final size check
try:
    db_size_bytes = os.path.getsize(db_path)
    db_size_mb = db_size_bytes / (1024 * 1024)
    print(f"Final database file size: {db_size_mb:.2f} MB")
except FileNotFoundError:
    print("Database file not found.")

Forcing database file rewrite to reclaim space...
Checkpoint complete.
Final database file size: 1722.51 MB
